### 2.2 NIO核心组件

#### 一. Channel
1. Channel与流相似但有以下几点不同: 
    * 流是单向的, 分为输入流和输出流. 但CHannel是双向的, 可读可写
    * Channel可以进行异步读写
    * Channel经常是read from或write to一个Buffer
2. Channel的几个实现
    * FileChannel: 读取或写入文件
    * DatagramChannel: 通过UDP读写数据
    * SocketChannel: 通过TCP读写数据
    * ServerSocketChannel: 向一个Server一样, 监听一个端口等待连接, 每接收一个链接, 都产生一个SocketChannel实例
    
#### 二. Buffer
1. Buffer的本质  
Buffer的本质是一个内存块(a block of memery),也就是数组,可以在这个区域上写入数据, 然后读取.这个内存块被包装成NIO Buffer对象, 以便读写 
2. Buffer的读写思路
    * 当写入数据到Buffer时,buffer会跟踪有多少数据已写入;一旦想要从中读取数据, 需要使用flip()方法, 把buffer从写模式切换成读模式.  
    * 当读完所有数据后, 需要使用clear()方法清空整个buffer, 做好准备再次进入"写模式";   
      或者使用compact()方法, 清空已经读完的数据, 并把未读取的数据移到buffer的开头; 再次进入写模式后在未读取的数据后面开始写
3. 使用buffer读写数据的4个步骤
    * Write data into the Buffer
    * Call buffer.flip(): 切换写模式->到读模式
    * Read data out of the Buffer
    * Call buffer.clear() or buffer.compact(): 切换读模式->到写模式
4. Buffer的capacity,position,limit概念
    * capacity:  
      因为Buffer本质是一个内存块, capacity就是这个内存块的大小. 一经初始化不能改变
    * position:
        * 当要读写数据时, 都要从position指定的位置开始读或写, 完成读写后更改position的指向, 以便下次读写
        * position的最大值为capacity-1
    * limit:
        * 写模式下, limit表示能写入多少数据, 其值等于capacity
        * 读模式下, limit表示可以从buffer中读取多少数据. 因此,当使用flip()切换写模式为读模式时, limit=当前的position, 表示已写入了多少数据
5. Buffer的操作
    * **分配内存**  
      如下分别可初始化一个48bytes的Buffer和128个字符的Buffer
      ```java
      ByteBuffer buf = ByteBuffer.allocate(48);
      CharBuffer buf = CharBuffer.allocate(1024);
      ```
    * **写入数据**  
      写数据有2种方式:
        * 从Chanel中写入数据
        ```java
        int bytesRead = inChannel.read(buf); //read into buffer.
        ```
        * 手动调用put()写入数据.put()有多种方式, 如制定position或写入一个数组
        ```java
        buf.put(127);    
        ```
    * **flip()**
        * `flip()`将Buffer从写模式切换成读模式
        * `flip()`把position置为0,把limit置为刚刚写完成后的position
    * **读数据**  
      读数据分为2中模式:
        * 把数据读到buffer中
        ```java
        //read from buffer into channel.
        int bytesWritten = inChannel.write(buf);
        ```
        * 使用get()手动读数据
        ```java
        //read from buffer into channel.
        byte aByte = buf.get();   
        ```
    * **rewind()**  
      将position置回0,limit不变,以便重新读取buffer中的数据
    * **clear()和compact()**
        * `clear()`
            * 将position置回0,limit置为capacity,已达到清空数据的目的
            * clear()方法并没有真正清空数据, 只是修改指针值并切换为写模式
        * `compact()`
            * 拷贝bufer中未读取的数据到buffer的头部, 并把position置为未读数据的右侧指针,limit置为capacity.以便从维度去的数据后面继续写入
    * **mark()和reset()**  
      `mark()`记录当前的position值,一顿读写操作后,`reset`可以将position置回mark时的位置
      ```java
      buffer.mark();
      //call buffer.get() a couple of times, e.g. during parsing.
      buffer.reset();  //set position back to mark.    
      ```

#### 三. 分散写和聚合读
1. scatter read和gather write   
Java NIO内建scatter/gather模式  
    * `scatter read`: 用于从Channel读数据到多个Buffer
    * `gather write`: 把数据从多个Buffer写入到Channel
2. scatter read from  
    * 将一个Channel中的数据依次读到多个Buffer中
    * 分散读意味着, 如果一个消息可以分为不同的part, 且每个part有固定的大小, 则可以使用分散度的方式将数据分散到自己应有的部分; 同时, 如果每个part的大小不固定, 则不应该使用scatter read
    <img src="img/scatter.png" height="30%" width="30%">
    * 如下指定一组Buffer:header和body,让Channel将数据按照Buffer在数组中的顺序依次读入, 读满一个Buffer则换到下一个Buffer读
    ```java
    ByteBuffer header = ByteBuffer.allocate(128);
    ByteBuffer body   = ByteBuffer.allocate(1024);

    ByteBuffer[] bufferArray = { header, body };

    channel.read(bufferArray);
    ```
3. gather write to
    <img src="img/gather.png" height="30%" width="30%">
    * 将多个Buffer中的数据依次写入到一个Channel
    * 如果一个Buffer有128bytes,但是其中只有56byte的数据, 则这个Buffer写入到channel的数据量是56byte
    ```java
    ByteBuffer header = ByteBuffer.allocate(128);
    ByteBuffer body   = ByteBuffer.allocate(1024);

    //write data into buffers

    ByteBuffer[] bufferArray = { header, body };

    channel.write(bufferArray);
    ```